# Rotenone Perturbation Rep 1 Extracting Topological Skeletons
In this notebook we are extracting topological skeletons in order to measure the distance along the medial axis. First we load the data and define a function so that we can process all of the images at the same time (making them the same size so we can stack them in numpy)

In [1]:
import os 
import skimage.io as io
import pyclesperanto_prototype as cle
import napari
import organoid_prediction_python as opp
import numpy as np

def make_images_equal_again(image_list):
    max_x = np.max([img.shape[1] for img in image_list])
    max_y = np.max([img.shape[0] for img in image_list])
    
    out_image_list = []
    
    for img in image_list:
        shape = img.shape
        if shape[0] < max_y:
            y_difference = max_y-shape[0]
            black_strip = np.full((y_difference, shape[1]),0)
            img = np.concatenate([black_strip,img],axis=0)
        if shape[1] < max_x:
            x_diff = max_x - shape[1]
            black_strip = np.full((max_y, x_diff),0)
            img = np.concatenate([black_strip,img],axis=1)
        out_image_list.append(img)
        
    return np.array(out_image_list)

home_directory = r"C:\Users\savill\OneDrive\Documents\PhD Jesse\Embryonic_organoid_prediction\Processing dataset for Paper\TLS_2D_morphostate_investigation/"
path = home_directory + "image_data/Processed Data/Perturbation Analysis/Mean Projections ROT Rep3/"

file_list = [file for file in os.listdir(path) if file.endswith("crop.tif")]
file_list[:5]

['72h_10nM_R_P1_14_01_crop.tif',
 '72h_10nM_R_P1_15_01_crop.tif',
 '72h_10nM_R_P1_16_01_crop.tif',
 '72h_10nM_R_P1_17_01_crop.tif',
 '72h_10nM_R_P1_18_01_crop.tif']

In [2]:
collection = io.imread_collection(path+"/*crop.tif")
collection.files[:5]

['C:\\Users\\savill\\OneDrive\\Documents\\PhD Jesse\\Embryonic_organoid_prediction\\Processing dataset for Paper\\TLS_2D_morphostate_investigation/image_data/Processed Data/Perturbation Analysis/Mean Projections ROT Rep3\\72h_10nM_R_P1_14_01_crop.tif',
 'C:\\Users\\savill\\OneDrive\\Documents\\PhD Jesse\\Embryonic_organoid_prediction\\Processing dataset for Paper\\TLS_2D_morphostate_investigation/image_data/Processed Data/Perturbation Analysis/Mean Projections ROT Rep3\\72h_10nM_R_P1_15_01_crop.tif',
 'C:\\Users\\savill\\OneDrive\\Documents\\PhD Jesse\\Embryonic_organoid_prediction\\Processing dataset for Paper\\TLS_2D_morphostate_investigation/image_data/Processed Data/Perturbation Analysis/Mean Projections ROT Rep3\\72h_10nM_R_P1_16_01_crop.tif',
 'C:\\Users\\savill\\OneDrive\\Documents\\PhD Jesse\\Embryonic_organoid_prediction\\Processing dataset for Paper\\TLS_2D_morphostate_investigation/image_data/Processed Data/Perturbation Analysis/Mean Projections ROT Rep3\\72h_10nM_R_P1_17_01

Binarising Images and taking a look in napari

In [3]:
import napari_simpleitk_image_processing as nsitk  # version 0.4.5

def processing(image):
    image1_gb = cle.gaussian_blur(image, None, 10.0, 10.0, 0.0)
    image2_to = cle.threshold_otsu(image1_gb)
    image3_B = nsitk.binary_fill_holes(image2_to)

    return image3_B

In [4]:
results = []
for image in collection:
    results.append(processing(image))

len(results)

297

In [5]:
viewer = napari.Viewer()
viewer.add_image(make_images_equal_again(collection))
viewer.add_labels(make_images_equal_again(results).astype(int))

<Labels layer 'Labels' at 0x199900dc340>

Keeping only one object and extracting image skeletons:

In [6]:
viewer.close()
only_one_object = opp.keep_labels_closest_to_stack_median(results)

In [7]:

import napari_segment_blobs_and_things_with_membranes as nsbatwm  # version 0.3.4
skeletons = [nsbatwm.skeletonize(label) for label in only_one_object]


In [8]:
viewer = napari.Viewer()
viewer.add_image(make_images_equal_again(collection))
viewer.add_labels(make_images_equal_again(skeletons).astype(int))

<Labels layer 'Labels' at 0x199cdd5f9a0>

In [9]:
home_directory = r"C:\Users\savill\OneDrive\Documents\PhD Jesse\Embryonic_organoid_prediction\Processing dataset for Paper\TLS_2D_morphostate_investigation/"
out_folder = home_directory + "image_data/Processed Data/Perturbation Analysis/Skeletons ROT Rep3/"

if not(os.path.isdir(out_folder)):
    os.mkdir(out_folder)

for image,name in zip(skeletons,collection.files):
    file_name = os.path.split(name)[1]
    io.imsave(out_folder+file_name[:-4] + "_skeleton.tif",image)